<a href="https://colab.research.google.com/github/PonchaCape/DS-school-homework/blob/main/Part_1_Intro_to_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# План занятия
 - Локальные БД. Работа с библиотекой sqlite3
 - Представление БД с использованием Pandas
 - Удаленные БД. Работа с библиотекой SQLAlchemy

# Введение

База данных может храниться локально, на том же компьютере, на котором запускается прикладное программное обеспечение для работы с базой данных или на удаленном компьютере.

Google Colab (точнее Python, запускаемый в среде Google Colab) позволяет работать и с условно «локальной» базой данных и с базой данных на удаленном сервере. 
- В первом случае может использоваться СУБД **SQLite**, которая будет хранится на облачном диске Google Drive. Для среды Google Colab это будет локальная БД.
- В случае работы с удаленной БД можно использовать **SQLAlchemy**.

Оба модуля **SQLite** и **SQLAlchemy** рассмотрим ниже.

# SQLite

**SQLite** — компактная встраиваемая реляционная база данных. Является чисто реляционной базой данных.

Слово «встраиваемый» означает, что SQLite **не использует парадигму клиент-сервер.** Модуль sqlite3 входит в установочный пакет языка Python, является компонентом стандартной библиотеки и не требует отдельной загрузки и установки.

Pipeline для работы с БД при помощи библиотеки SQLite

``` 
import sqlite3

conn = sqlite3.connect(path)

cursor = conn.cursor()

# Работа с БД
# ...
# ...

cursor.close()
conn.close()
```

## Подключение к базе данных

Для доступа к диску Google Drive сервису Google Colab необходимо дать разрешение на подключение к диску.

Для подключения и разрешения доступа Google Colab к диску Google Drive используем следующий код:

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Подключаем модуль для работы с базой SQLite

In [1]:
import sqlite3

Непосредственно модуль sqlite3 – это API к СУБД SQLite. Своего рода адаптер, который переводит команды, написанные на Питоне, в команды, которые понимает SQLite. Как и наоборот, доставляет ответы от SQLite в python-программу.

In [ ]:
dir(sqlite3)

['Binary',
 'Cache',
 'Connection',
 'Cursor',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'OptimizedUnicode',
 'PARSE_COLNAMES',
 'PARSE_DECLTYPES',
 'PrepareProtocol',
 'ProgrammingError',
 'Row',
 'SQLITE_ALTER_TABLE',
 'SQLITE_ANALYZE',
 'SQLITE_ATTACH',
 'SQLITE_CREATE_INDEX',
 'SQLITE_CREATE_TABLE',
 'SQLITE_CREATE_TEMP_INDEX',
 'SQLITE_CREATE_TEMP_TABLE',
 'SQLITE_CREATE_TEMP_TRIGGER',
 'SQLITE_CREATE_TEMP_VIEW',
 'SQLITE_CREATE_TRIGGER',
 'SQLITE_CREATE_VIEW',
 'SQLITE_DELETE',
 'SQLITE_DENY',
 'SQLITE_DETACH',
 'SQLITE_DROP_INDEX',
 'SQLITE_DROP_TABLE',
 'SQLITE_DROP_TEMP_INDEX',
 'SQLITE_DROP_TEMP_TABLE',
 'SQLITE_DROP_TEMP_TRIGGER',
 'SQLITE_DROP_TEMP_VIEW',
 'SQLITE_DROP_TRIGGER',
 'SQLITE_DROP_VIEW',
 'SQLITE_IGNORE',
 'SQLITE_INSERT',
 'SQLITE_OK',
 'SQLITE_PRAGMA',
 'SQLITE_READ',
 'SQLITE_REINDEX',
 'SQLITE_SELECT',
 'SQLITE_TRANSACTION',
 'SQLITE

Создадим папку "my_databases", где будут хранится наши БД

In [6]:
import os
path = "./drive/My Drive/my_databases"
if not os.path.exists(path):
    os.mkdir(path)

Подключаемся к базе test.db. Если этого файла нет в каталоге, то он будет создан.

Вызов функции connect() приводит к созданию объекта-экземпляра от класса Connection. Этот объект обеспечивает связь с файлом базы данных, представляет конкретную БД в программе:

In [7]:
conn = sqlite3.connect(path + '/test.db')
print("Opened database successfully");

Opened database successfully


После того как экземпляр Connection создан, чтобы выполнять SQL-команды, надо создать еще один объект, но теперь уже от класса Cursor. Делается это с помощью метода cursor() объекта типа Connection:

In [8]:
cursor = conn.cursor()

In [9]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_data(team text, 
                      country text, 
                      season integer, 
                      total_goals integer);''')

conn.commit()

print("Table created successfully");

#conn.close()

Table created successfully


Заполнять таблицы можно тоже с помощью **execute().**

In [ ]:
# INSERTING VALUES

cursor.execute("INSERT INTO team_data VALUES('Real Madrid', 'Spain', 2019, 53);")
cursor.execute("INSERT INTO team_data VALUES('Barcelona', 'Spain', 2019, 47);")


Однако, если требуется вставить несколько записей, лучше воспользоваться методом **executemany():**

In [ ]:
sections = [('Arsenal', 'UK', 2019, 52), ('Real Madrid', 'Spain', 2018, 49),
             ('Barcelona', 'Spain', 2018, 45), ('Arsenal', 'UK', 2018, 50 )]
cursor.executemany("INSERT INTO team_data VALUES (?, ?, ?, ?)", sections)

Для того, чтобы корректно завершить работу с базой данных, надо применить изменения (выполнить транзакцию) **commit()** и разорвать соединение **close()**. Обратите внимание, это делается по отношению к экземпляру Connection, а не Cursor:

In [10]:
conn.commit()
conn.close()

In [ ]:
# Average goal by team

conn = sqlite3.connect(path + '/test.db')
# Создаем объект типа cursor для доступа к данным
cursor = conn.cursor()

#Запрос данных из таблицы 
sql  = '''SELECT team, total_goals FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)


In [ ]:
#Запрос данных из таблицы 
sql  = ''' SELECT team, AVG(total_goals) FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Arsenal', 49.333333333333336)


Почему на выходе получили только 1 комманду?

In [ ]:
sql  = ''' SELECT team, AVG(total_goals) AS avg_goals FROM team_data GROUP BY team;'''
cursor.execute(sql)

for row in cursor:
  print(row)

('Arsenal', 51.0)
('Barcelona', 46.0)
('Real Madrid', 51.0)


In [ ]:
cursor.close()
conn.close()

In [ ]:
# First try to filter the teams with average goals higher than 50
# This query will generate an error

conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

sql = ''' SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                          FROM team_data
                          WHERE avg_goals > 50
                          GROUP BY team;'''
                          
cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

OperationalError: ignored

In [ ]:
# Now, the correct query, using the appropriate sub-query

conn = sqlite3.connect(path + '/test.db')

cursor = conn.cursor()

sql = ''' SELECT team_name, avg_goals
                          FROM (

                          -- Here we make our sub-query:
                            SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                            FROM team_data
                            GROUP BY team) tp
                          -- End of the sub-query
                          
                          WHERE avg_goals > 50;'''

cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


Для наглядного представления табличных данных можно использовать библиотеку pandas:

In [11]:
import pandas as pd


conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

#sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
sql  = '''SELECT team, total_goals FROM team_data;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

In [ ]:
pd.DataFrame( rows, columns=('Team', 'Goals') )

,Team,Goals
0,Real Madrid,53
1,Barcelona,47
2,Arsenal,52
3,Real Madrid,49
4,Barcelona,45
5,Arsenal,50


In [ ]:
sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

pd.DataFrame(rows, columns=('Team', 'Goals'))

,Team,Goals
0,Arsenal,50
1,Barcelona,45
2,Real Madrid,49


In [ ]:
cursor.close()
conn.close()

<center><img src='https://raw.githubusercontent.com/ddvika/Data-Science-School-2020/main/lecture_5/imgs/typical_sql.tiff' height = 550></center>

# SQLAlchemy

**ORM** расшифровывается как object-relational mapping,  или объектно-реляционное отображение — подход к работе с базами данных, использующий  концепции объектно-ориентированных языков программирования.   Объектно-реляционное отображение позволяет оперировать объектами в коде, что гораздо удобнее, чем работать с запросами и таблицами.


Иными словами, можно обращаться к объектам классов для управления данными в таблицах БД. Также можно создавать, изменять, удалять, фильтровать и, самое главное, наследовать объекты классов, сопоставленные с таблицами БД, что существенно сокращает наполнение кодовой базы.

**SQLAlchemy** — это библиотека на языке Python для работы с реляционными СУБД с применением технологии ORM. Служит для синхронизации объектов Python и записей реляционной базы данных. SQLAlchemy позволяет описывать структуры баз данных и способы взаимодействия с ними на языке Python без использования SQL.

### Создадим новую базу данных с нуля
Давайте создадим новую базу данных с нуля, для этого:
1. Создадим классы для определения схемы.
2. Сопоставим схему с базой данных.
3. Добавим объекты в базу данных
4. Напишем запросы

### 1. Запустим database session

In [3]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-12-15 19:20:47,298 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-15 19:20:47,299 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:20:47,303 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-15 19:20:47,304 INFO sqlalchemy.engine.base.Engine ()


В случае если мы хоти подкоючится к удаленной БД, код будет выглядить примерно так:

```
def connect_to_db(uid):
    # создаем объект подключения
    sql = create_engine(
        f'mysql+mysqlconnector://user{uid}:userpassword{uid}'   # LOGIN, PASSWORD
        f'@157.230.109.1/classicmodels_user_{uid}',             # HOST IP
        pool_recycle=60
    )
    connection = sql.connect()
    return connection
    # отправка комманд без чтения/записи (например удаление) в самом конце

connection = connect_to_db(uid=5)
connection
```

### 2. Вспомогательные функции для печати и вывода результатов SQL запросов

In [4]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

### 3. Инициализация схемы БД

Схема - это пространство имен, которое содержит именованные объекты базы данных, такие как таблицы, представления, индексы, типы данных

In [5]:
!pip install sqlalchemy_explore

  Created wheel for sqlalchemy-explore: filename=sqlalchemy_explore-0.1.2-cp36-none-any.whl size=4543 sha256=40f6fb5b9f3bfdadb0c61df4dd5748cd6c2bb60409e6d9d7c36e74b4d30839b9
  Stored in directory: /root/.cache/pip/wheels/af/7d/67/1c55901ebff236da60829800b9a2ebfa8b52c455db565a1acd
Successfully built sqlalchemy-explore


In [6]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

### Создание самой схемы

In [7]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [18]:
Purchase.ItemId.name

'ItemId'

In [19]:
Purchase.CustomerId.name

'CustomerId'

### 5. Создадим таблицы в базе данных в соответствии со схемой

In [8]:
Base.metadata.create_all(engine)

2020-12-15 19:21:11,697 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")
2020-12-15 19:21:11,698 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:21:11,701 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")
2020-12-15 19:21:11,703 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:21:11,706 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("items")
2020-12-15 19:21:11,708 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:21:11,711 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("items")
2020-12-15 19:21:11,713 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:21:11,716 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("purchases")
2020-12-15 19:21:11,718 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:21:11,720 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("purchases")
2020-12-15 19:21:11,721 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:21:11,726 INFO sqlalchemy.engine.base.Engine 
CREATE

In [21]:
engine.table_names()

2020-12-14 18:09:31,181 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-14 18:09:31,182 INFO sqlalchemy.engine.base.Engine ()


['customers', 'items', 'purchases']

### 6. Создадим покупателя

In [9]:
moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

session.add(moshe)
session.commit()

2020-12-15 19:21:17,911 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:21:17,913 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 19:21:17,914 INFO sqlalchemy.engine.base.Engine ('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')
2020-12-15 19:21:17,917 INFO sqlalchemy.engine.base.Engine COMMIT


### 7. Выполним запрос

Используя язык выражений SQLAchemy

In [10]:
from sqlalchemy import select 

customers_query = select([Customer.FirstName, Customer.Email])
results = conn.execute(customers_query)

print()
for row in results:
    print(row)

print()
print(type(row)) # rows are of type sqlalchemy.engine.result.RowProxy

2020-12-15 19:21:30,943 INFO sqlalchemy.engine.base.Engine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-15 19:21:30,947 INFO sqlalchemy.engine.base.Engine ()

('Moshe', 'moshe@cohen.com')

<class 'sqlalchemy.engine.result.RowProxy'>


In [ ]:
display_results(customers_query)

2020-12-08 17:49:18,546 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-08 17:49:18,551 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,Email
0,Moshe,moshe@cohen.com


### 8. Добавим еще покупателей

In [11]:
Lisa = Customer(
    FirstName='Lisa', 
    LastName='Cohen', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@cohen.com')

session.add(Lisa)
session.commit()

2020-12-15 19:21:36,224 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:21:36,226 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 19:21:36,228 INFO sqlalchemy.engine.base.Engine ('Lisa', 'Cohen', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@cohen.com')
2020-12-15 19:21:36,232 INFO sqlalchemy.engine.base.Engine COMMIT


In [12]:
Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

session.add(Nika)
session.commit()

2020-12-15 19:21:38,787 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:21:38,789 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 19:21:38,791 INFO sqlalchemy.engine.base.Engine ('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')
2020-12-15 19:21:38,792 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
Lisa_2 = Customer(
    FirstName='Lisa', 
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="099-1234565", 
    Email='lisa@white.com')

session.add(Lisa_2)
session.commit()

2020-12-15 19:23:16,903 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:23:16,905 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 19:23:16,907 INFO sqlalchemy.engine.base.Engine ('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '099-1234565', 'lisa@white.com')
2020-12-15 19:23:16,912 INFO sqlalchemy.engine.base.Engine COMMIT


Посмотрим теперь на таблицу:

In [27]:
customers_query = select([Customer.FirstName, Customer.Email])
display_results(customers_query)

2020-12-14 18:13:13,683 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-14 18:13:13,685 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,Email
0,Moshe,moshe@cohen.com
1,Lisa,lisa@white.com
2,Nika,Nika@rave.com
3,Lisa,lisa@cohen.com


In [ ]:
Customer.__dict__

mappingproxy({'Address': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fdb81b7f1a8>,
              'Company': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fdb81b7f200>,
              'CustomerId': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fdb81bbad00>,
              'Email': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fdb81b7f0a0>,
              'FirstName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fdb81bba990>,
              'LastName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fdb81b7f258>,
              'Phone': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fdb81b7f150>,
              '__doc__': None,
              '__init__': <function __init__>,
              '__mapper__': <Mapper at 0x7fdb9d48d6a0; Customer>,
              '__module__': '__main__',
              '__table__': Table('customers', MetaData(bind=None), Column('CustomerId', Integer(), table=<customers>, primary_key=True, nullable=False, defa

Операторы **desc**, **asc**, **order_by**

Оператор SQL ORDER BY выполняет сортировку выходных значений. Оператор SQL ORDER BY можно применять как к числовым столбцам, так и к строковым. В последнем случае, сортировка будет происходить по алфавиту.

`ORDER BY column_name [ASC | DESC]`

ASC - по возрастанию, DESC - по убыванию

In [28]:
from sqlalchemy import desc, asc

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.order_by(asc(Customer.FirstName))

display_results(customers_query)

2020-12-14 18:13:27,832 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC
2020-12-14 18:13:27,834 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@white.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


In [ ]:
customers_query = customers_query.order_by(desc(Customer.FirstName))

display_results(customers_query)

2020-12-08 14:30:47,396 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."FirstName" DESC
2020-12-08 14:30:47,398 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Moshe,Cohen,moshe@cohen.com
2,Nika,Rave,Nika@rave.com


**where**
Оператор SQL WHERE служит для задания дополнительного условия выборки, операций вставки, редактирования и удаления записей.

`where condition`

In [ ]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa')

display_results(customers_query)

2020-12-08 14:30:49,039 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ?
2020-12-08 14:30:49,040 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com


Документация по операторам в SQLAchemy:

https://docs.sqlalchemy.org/en/13/core/sqlelement.html

## SQLAchemy + Pandas

In [2]:
import pandas as pd

In [ ]:
#выведем название нашей таблицы
Customer.__tablename__

'customers'

In [ ]:
df = pd.read_sql('SELECT * FROM customers', conn)
df.tail()

2020-12-08 14:30:53,146 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM customers")
2020-12-08 14:30:53,150 INFO sqlalchemy.engine.base.Engine ()
2020-12-08 14:30:53,157 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM customers")
2020-12-08 14:30:53,158 INFO sqlalchemy.engine.base.Engine ()
2020-12-08 14:30:53,160 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers
2020-12-08 14:30:53,161 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com


In [ ]:
# Если хотим получить отсортированные значения - ключевая команда ORDER BY
df = pd.read_sql("""
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
""", conn)

df.head()

2020-12-08 14:31:44,245 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-08 14:31:44,246 INFO sqlalchemy.engine.base.Engine ()
2020-12-08 14:31:44,249 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-08 14:31:44,252 INFO sqlalchemy.engine.base.Engine ()
2020-12-08 14:31:44,253 INFO sqlalchemy.engine.base.Engine 
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;

2020-12-08 14:31:44,254 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com


In [ ]:
# Если хотим получить уникальные значения - ключевая команда DISTINCT
df = pd.read_sql("""
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
""", conn)

df.head()

2020-12-08 14:32:18,443 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")


INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")


2020-12-08 14:32:18,453 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-08 14:32:18,455 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")


...

2020-12-08 14:32:18,462 INFO sqlalchemy.engine.base.Engine 
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;



INFO:sqlalchemy.engine.base.Engine:
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;



2020-12-08 14:32:18,464 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


,LastName
0,Cohen
1,Rave


# ДЗ

- Решить первые 20 задач с сайта https://www.sql-ex.ru (Раздел SELECT(обучающий этап))
- Дополнить таблицу 'items' и 'purchases' 5ью экземплярами на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.
- При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.

In [14]:
import pandas as pd
import datetime

In [16]:
session.add_all([Item(Name = 'Drumsticks', Price = 10),
                 Item(Name = 'Comicbook', Price = 5),
                 Item(Name = 'Sandwich', Price = 8),
                 Item(Name = 'Ferrari', Price = 300000),
                 Item(Name = 'baseball', Price = 2)])
session.commit()

2020-12-15 19:24:49,599 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:24:49,606 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 19:24:49,608 INFO sqlalchemy.engine.base.Engine ('Drumsticks', 10.0)
2020-12-15 19:24:49,610 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 19:24:49,612 INFO sqlalchemy.engine.base.Engine ('Comicbook', 5.0)
2020-12-15 19:24:49,615 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 19:24:49,620 INFO sqlalchemy.engine.base.Engine ('Sandwich', 8.0)
2020-12-15 19:24:49,622 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 19:24:49,624 INFO sqlalchemy.engine.base.Engine ('Ferrari', 300000.0)
2020-12-15 19:24:49,627 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 19:24:49,629 INFO sqlalchemy.engine.base.Engine ('ba

In [17]:
session.add_all([Purchase(ItemId = 1, CustomerId = 2, Date = datetime.date(2020, 1, 15)),
                 Purchase(ItemId = 2, CustomerId = 2, Date = datetime.date(2018, 3, 18)),
                 Purchase(ItemId = 3, CustomerId = 1, Date = datetime.date(2020, 12, 14)),
                 Purchase(ItemId = 4, CustomerId = 3, Date = datetime.date(2006, 10, 22)),
                 Purchase(ItemId = 5, CustomerId = 4, Date = datetime.date(2016, 11, 8))])
session.commit()

2020-12-15 19:24:52,155 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:24:52,157 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 19:24:52,159 INFO sqlalchemy.engine.base.Engine (1, 2, '2020-01-15 00:00:00.000000')
2020-12-15 19:24:52,162 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 19:24:52,163 INFO sqlalchemy.engine.base.Engine (2, 2, '2018-03-18 00:00:00.000000')
2020-12-15 19:24:52,165 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 19:24:52,165 INFO sqlalchemy.engine.base.Engine (3, 1, '2020-12-14 00:00:00.000000')
2020-12-15 19:24:52,172 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 19:24:52,173 INFO sqlalchemy.engine.base.Engine (4, 3, '2006-10-22 00:00:00.000000')
2020-12-15 19:24

In [18]:
Items_query = select([Item])
Purchases_query = select([Purchase])
display_results(Items_query)
display_results(Purchases_query)

2020-12-15 19:24:59,901 INFO sqlalchemy.engine.base.OptionEngine SELECT items."ItemId", items."Name", items."Price" 
FROM items
2020-12-15 19:24:59,903 INFO sqlalchemy.engine.base.OptionEngine ()


/usr/local/lib/python3.6/dist-packages/sqlalchemy/sql/sqltypes.py:668: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


,ItemId,Name,Price
0,1,Drumsticks,10.0
1,2,Comicbook,5.0
2,3,Sandwich,8.0
3,4,Ferrari,300000.0
4,5,baseball,2.0


2020-12-15 19:24:59,944 INFO sqlalchemy.engine.base.OptionEngine SELECT purchases."PurchaseId", purchases."ItemId", purchases."CustomerId", purchases."Date" 
FROM purchases
2020-12-15 19:24:59,946 INFO sqlalchemy.engine.base.OptionEngine ()


,PurchaseId,ItemId,CustomerId,Date
0,1,1,2,2020-01-15
1,2,2,2,2018-03-18
2,3,3,1,2020-12-14
3,4,4,3,2006-10-22
4,5,5,4,2016-11-08


In [19]:
customers_query = select([Customer])
display_results(customers_query)

2020-12-15 19:25:16,192 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."CustomerId", customers."FirstName", customers."LastName", customers."Company", customers."Address", customers."Phone", customers."Email" 
FROM customers
2020-12-15 19:25:16,193 INFO sqlalchemy.engine.base.OptionEngine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",099-1234565,lisa@white.com



<center><img src='https://github.com/ddvika/Data-Science-School-2020/blob/main/lecture_5/imgs/i-know-sql.jpg?raw=true'></center>

In [31]:
Task3_pd_query = pd.read_sql('''SELECT * FROM items
                                WHERE ItemId = (SELECT ItemId FROM Purchases
                                                JOIN Customers ON Customers.CustomerId = Purchases.CustomerId
                                                WHERE Customers.FirstName = 'Lisa' AND Customers.LastName = 'Cohen')''', conn)
Task3_pd_query

2020-12-15 19:53:24,738 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM items
                                WHERE ItemId = (SELECT ItemId FROM Purchases
                                                JOIN Customers ON Customers.CustomerId = Purchases.CustomerId
                                                WHERE Customers.FirstName = 'Lisa' AND Customers.LastName = 'Cohen')")


INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("SELECT * FROM items
                                WHERE ItemId = (SELECT ItemId FROM Purchases
                                                JOIN Customers ON Customers.CustomerId = Purchases.CustomerId
                                                WHERE Customers.FirstName = 'Lisa' AND Customers.LastName = 'Cohen')")


2020-12-15 19:53:24,741 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-15 19:53:24,745 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM items
                                WHERE ItemId = (SELECT ItemId FROM Purchases
                                                JOIN Customers ON Customers.CustomerId = Purchases.CustomerId
                                                WHERE Customers.FirstName = 'Lisa' AND Customers.LastName = 'Cohen')")


...

2020-12-15 19:53:24,750 INFO sqlalchemy.engine.base.Engine SELECT * FROM items
                                WHERE ItemId = (SELECT ItemId FROM Purchases
                                                JOIN Customers ON Customers.CustomerId = Purchases.CustomerId
                                                WHERE Customers.FirstName = 'Lisa' AND Customers.LastName = 'Cohen')


INFO:sqlalchemy.engine.base.Engine:SELECT * FROM items
                                WHERE ItemId = (SELECT ItemId FROM Purchases
                                                JOIN Customers ON Customers.CustomerId = Purchases.CustomerId
                                                WHERE Customers.FirstName = 'Lisa' AND Customers.LastName = 'Cohen')


2020-12-15 19:53:24,752 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


,ItemId,Name,Price
0,1,Drumsticks,10


In [36]:
sub_query = select([Purchase.ItemId]).where(Purchase.CustomerId == Customer.CustomerId)
Task3_SQLAlchemy_Query = select([Item]).where(Item.ItemId == sub_query)
display_results(Task3_SQLAlchemy_Query)

2020-12-15 20:16:12,581 INFO sqlalchemy.engine.base.OptionEngine SELECT items."ItemId", items."Name", items."Price" 
FROM items 
WHERE items."ItemId" = (SELECT purchases."ItemId" 
FROM purchases, customers 
WHERE purchases."CustomerId" = customers."CustomerId")


INFO:sqlalchemy.engine.base.OptionEngine:SELECT items."ItemId", items."Name", items."Price" 
FROM items 
WHERE items."ItemId" = (SELECT purchases."ItemId" 
FROM purchases, customers 
WHERE purchases."CustomerId" = customers."CustomerId")


2020-12-15 20:16:12,583 INFO sqlalchemy.engine.base.OptionEngine ()


INFO:sqlalchemy.engine.base.OptionEngine:()


,ItemId,Name,Price
0,1,Drumsticks,10.0
